In [2]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeamsInQuery, getSeasonsInQuery, getStagesInQuery
import sqlAnswers

In [3]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [4]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    
    if(query[-1:] == '?' or query[-1:] == '!' or query[-1:] == '.' ):
        return query[:-1]
    return query

Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [6]:
def getTeamMatches(): #testausrufe
    import sqlite3
    conn = sqlite3.connect('database.sqlite')
    c = conn.cursor()
    queryText = 'SELECT * FROM match WHERE id = 307'
    c.execute(queryText)
    result = c.fetchall()
    conn.close()
    return result
print(getTeamMatches())

[(307, 1, 1, '2009/2010', 1, '2009-07-31 00:00:00', 665318, 9985, 9997, 2, 2, 1, 2, 4, 6, 8, 2, 4, 6, 8, 4, 6, 1, 2, 4, 6, 8, 2, 4, 6, 8, 4, 6, 1, 3, 3, 3, 3, 7, 7, 7, 7, 10, 10, 1, 3, 3, 3, 3, 7, 7, 7, 7, 10, 10, 38797, 39580, 38800, 37861, 47411, 35412, 39631, 39591, 37262, 25957, 38369, 37868, 33595, 38347, 3329, 149150, 38354, 37866, 36836, None, 5016, 45490, None, None, None, None, None, None, None, None, 1.22, 6, 13, 1.2, 5.5, 10.5, 1.2, 5, 10, 1.17, 5.5, 12, None, None, None, 1.2, 5, 11, 1.18, 6.5, 13, 1.2, 5.5, 12, 1.2, 5.5, 11, 1.17, 6, 11)]


In [7]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
german_embedding = WordEmbeddings('de-crawl')
 
import numpy
import json

In [8]:
def getVectorOfAnswer(index):
    with open('trainData.json', 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    embeddings = 0
    for sentence in obj[str(index)]:
        
        sentence = Sentence(preProcessQuery(sentence))
        german_embedding.embed(sentence)
        
        embed = 0
        for token in sentence:
            #print(token)
            embed += token.embedding.numpy()
        #evtl embed durch die anzahl an Wörtern darin teilen ? beim Finalen auch?
        embeddings += embed
    return embeddings
#print(getVectorOfAnswer(0))

In [9]:
import numpy as np
def getVectorOfQuery(query):
    sentence = Sentence(query)
    german_embedding.embed(sentence)
    
    embed = 0
    for token in sentence:
        #print(token)
        embed += token.embedding.numpy()
    return embed


def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



In [27]:
def getAnswer(question):
    query = preProcessQuery(question)
    
    pQ = getPlayersInQuery(query)
    lQ = getLeaguesInQuery(pQ[1])
    tQ = getTeamsInQuery(lQ[1])
    sQ = getSeasonsInQuery(tQ[1])
    yQ = getStagesInQuery(sQ[1])
    print("Query, aus dem der Vektor gemacht wird:")
    print(yQ[1]) # die letzt query die dann auch dem vector gegeben wird
    print("")
    queryV = getVectorOfQuery(yQ[1]) # erwarter preprocessed Query, TODO 
    
    queryRefined = [pQ[0],lQ[0],tQ[0],sQ[0],yQ[0]]
    #queryV = getVectorOfQuery(pQ[1]) # erwarter preprocessed Query, TODO 
    #print(getPlayersInQuery(query)[1])

    #queryRefined = [pQ[0], getLeaguesInQuery(query),getTeamsInQuery(query),getSeasonsInQuery(query),getStagesInQuery(query)]
    print("gefundene Dinge: ")
    print(queryRefined)
    answers = sqlAnswers.getAnswers()


    queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3]),len(queryRefined[4])]
    possibleAnswers = [] 
    #contains index with possible ansers which could match the query depending
    #on number of player occurances etc

    for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
        sqlMatches = True;
        for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
            if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
                sqlMatches = False
        if(sqlMatches):
            possibleAnswers.append(x)
        

    print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
    print(possibleAnswers)
    if(len(possibleAnswers) == 1):
        return answers[possibleAnswers[0]][5](queryRefined)
    if(len(possibleAnswers) == 0):
        return "Es konnte leider keine passende Antwort gefunden werden!"
    angles = []
    for answerIndex in possibleAnswers:
        angles.append(angle_between(getVectorOfAnswer(answerIndex),queryV))

    print(angles)
    print("Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: " + str(angles.index(min(angles))))

    return answers[angles.index(min(angles))][5](queryRefined)

In [39]:
#Stellen Sie Ihre Frage:
eingabe = input("Ihre Eingabe? ")
print(getAnswer(eingabe))

Ihre Eingabe? Wo steht Manuel Neuer unter Vertrag?
Query, aus dem der Vektor gemacht wird:
Wo steht SPIELER unter Vertrag

gefundene Dinge: 
[[6556], [], [], [], []]
In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[0, 1, 2, 3]
[0.7824522, 0.85095006, 0.8254116, 0.3808763]
Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: 3
Der Spieler spielte in Saison 2015/2016 bei FC Bayern Munich.


Manuel Neuer und Thomas Müller 2009 Bundesliga
Wo spielte Thomas Müller Heimverein?
Was ist der Heimverein von Manuel Neuer?
Was ist der Heimverein von Toni Kroos?

Wo hat Manuel Neuer 2008/2009 gespielt?
Wo hat Manuel Neuer 2014/2015 gespielt?

was ist der bevorzugte Fuß von Toni Kroos?

Was ist das Geburtsdatum von Jerome Boateng?

In [11]:
print(getVectorOfQuery("Verein"))

[ 0.0101 -0.0324  0.0064  0.0767  0.0522 -0.0582  0.015  -0.0181 -0.0356
 -0.0003 -0.0372 -0.063  -0.0258  0.0188  0.0481 -0.037  -0.0125 -0.1202
 -0.056  -0.0276 -0.0389 -0.0457 -0.0366  0.0036  0.0618  0.0006 -0.0593
 -0.0095 -0.015   0.0026 -0.0403  0.0349  0.0058  0.     -0.0371  0.0427
 -0.0589  0.0689  0.0384 -0.0068 -0.0181 -0.0502  0.0126 -0.0265 -0.0411
  0.0172  0.0711  0.0301  0.0403 -0.0197 -0.0302 -0.0847 -0.0432 -0.0283
 -0.0503  0.0651  0.0048 -0.0152 -0.0291  0.0245  0.0095  0.0865  0.0317
  0.0546  0.0408  0.0445  0.0669 -0.0715  0.0694 -0.0169  0.0126 -0.0683
 -0.0098 -0.0072 -0.0273  0.0328 -0.0389  0.0533 -0.0303 -0.0352 -0.0482
 -0.0097 -0.048  -0.014  -0.0036  0.0726  0.0089 -0.0113 -0.0135  0.0309
  0.0325 -0.0185  0.0069  0.0055  0.08   -0.0394  0.0544  0.0144 -0.0015
 -0.0202 -0.0241 -0.0654 -0.025  -0.0284  0.0439  0.0265 -0.0031  0.004
 -0.0293 -0.0101  0.0059 -0.0386 -0.0235 -0.0442  0.0452  0.0019  0.0396
 -0.0618  0.0229 -0.04   -0.0108 -0.0165  0.0409 -0.